In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
import joblib

In [ ]:
DATA_PATH = 'emails.csv'  # Replace with your actual path
MODEL_PATH = 'phishing_model.pkl'

In [ ]:
##################################################
## @brief Iterate through files in a dir and aggregate them to a pandas dataframe 
## @in string           base_folder_path
## @out pd.DataFrame    df
##################################################
def load_and_prep_data(path):
    """
    Loads data and performs a stratified split to maintain class ratios.
    """
    print("[...] Loading Data")
    # Generating dummy data for demonstration if you run this code immediately
    # Remove this block when using your real CSV
    data = {
        'text': [
            "Hey, are we still on for lunch?", 
            "Please review the attached quarterly report.", 
            "URGENT: Your account has been compromised. Click here to reset.", 
            "Win a free iPhone now! Claim your prize.",
            "Meeting reminder: Project Alpha sync at 2 PM.",
            "Verify your bank details immediately to avoid suspension."
        ] * 50,
        'label': [0, 0, 1, 1, 0, 1] * 50 # 0: Ham, 1: Spam
    }
    df = pd.DataFrame(data)
    # df = pd.read_csv(path) # Uncomment for real usage
    
    X = df['text']
    y = df['label']
    
    return train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)